In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
from tensorflow.keras import layers, models
import sklearn.metrics as metrics

In [5]:
import pyodbc
import pandas as pd
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '' 
database = '' 
username = '' 
password = ''  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
query = "SELECT [GEOPHONE-Z],[TIMESTAMP],[Class] FROM dbo.FINAL;"
df = pd.read_sql(query, cnxn)
print(df.head(26))
df.shape

C:\Users\Leo\AppData\Local\Temp\ipykernel_15672\2310543218.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


    GEOPHONE-Z            TIMESTAMP  Class
0    0,0042886  2023-04-04 15:00:00  noise
1    0,0028228  2023-04-04 15:00:00  noise
2    0,0051194  2023-04-04 15:00:00  noise
3    0,0052139  2023-04-04 15:00:00  noise
4     0,002509  2023-04-04 15:00:00  noise
5    0,0033907  2023-04-04 15:00:00  noise
6    0,0010638  2023-04-04 15:00:00  noise
7    0,0004011  2023-04-04 15:00:00  noise
8   -0,0015425  2023-04-04 15:00:00  noise
9    -0,002916  2023-04-04 15:00:00  noise
10  -0,0022521  2023-04-04 15:00:00  noise
11  -0,0004765  2023-04-04 15:00:00  noise
12   0,0003961  2023-04-04 15:00:00  noise
13   0,0018307  2023-04-04 15:00:00  noise
14   0,0018996  2023-04-04 15:00:00  noise
15   0,0003917  2023-04-04 15:00:00  noise
16  -0,0001697  2023-04-04 15:00:00  noise
17  -0,0020734  2023-04-04 15:00:00  noise
18  -0,0029513  2023-04-04 15:00:00  noise
19  -0,0012475  2023-04-04 15:00:00  noise
20   -0,000175  2023-04-04 15:00:00  noise
21  -0,0006606  2023-04-04 15:00:00  noise
22  -0,0006

(1481189, 3)

In [10]:
df.drop(['TIMESTAMP'], axis=1 ,inplace=True)

In [11]:
df.rename(columns={'GEOPHONE-Z' : 'Value'},inplace=True)

In [17]:
df['Class'].value_counts()

Class
noise    1160095
car       321094
Name: count, dtype: int64

In [22]:
df['Value'] = df['Value'].str.replace(',', '.').astype('float32')


In [23]:
df["Class"].replace({"noise": 0, "car": 1}, inplace=True)


In [24]:
min_val = df['Value'].min()
max_val = df['Value'].max()

df['NORMALIZED'] = ((df['Value'] - min_val) / (max_val - min_val)) * 2 - 1



In [26]:
df.drop('Value', axis=1, inplace=True)

In [29]:
X=df.drop(['Class'],axis=1)
y=df['Class']

In [30]:
import pandas as pd
from collections import Counter
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Summarize class distribution
counter = Counter(y)
print(counter)

# Define the undersampling method
undersample = NearMiss(version=1, n_neighbors=3)

# Undersample the dataset
X_res, y_res = undersample.fit_resample(X, y)

# Summarize the new class distribution
counter_resampled = Counter(y_res)
print(counter_resampled)


Counter({0: 1160095, 1: 321094})
Counter({0: 321094, 1: 321094})


In [31]:
print (X_res.shape)
print(y_res.shape)

(642188, 1)
(642188,)


In [32]:
df

,Class,NORMALIZED
0,0,0.312586
1,0,0.310151
2,0,0.313965
3,0,0.314122
4,0,0.309630
...,...,...
1481184,0,0.301707
1481185,0,0.304831
1481186,0,0.306744
1481187,0,0.309069


In [33]:
df.value_counts

<bound method DataFrame.value_counts of          Class  NORMALIZED
0            0    0.312586
1            0    0.310151
2            0    0.313965
3            0    0.314122
4            0    0.309630
...        ...         ...
1481184      0    0.301707
1481185      0    0.304831
1481186      0    0.306744
1481187      0    0.309069
1481188      0    0.310375

[1481189 rows x 2 columns]>

In [34]:
X_train,X_test,y_train,y_test=train_test_split (X_res,y_res, test_size=0.2, random_state=42)


In [35]:
model_car = models.Sequential(name='model_car')
model_car.add(layers.LSTM(64, input_shape=(X_train.shape[1], 1),return_sequences=True))
model_car.add(layers.Flatten())
model_car.add(layers.Dense(32, activation='relu'))
model_car.add(layers.Dense(16, activation='relu'))
model_car.add(layers.Dense(1, activation='sigmoid'))

In [36]:
model_car.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [37]:
history=model_car.fit(X_train, y_train, batch_size=64, epochs=4)

y_pred = model_car.predict(X_test)

# Convert the predictions to binary format
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate evaluation metrics on the test set
precision = metrics.precision_score(y_test, y_pred_binary, zero_division=0)
recall = metrics.recall_score(y_test, y_pred_binary)
f1_score = metrics.f1_score(y_test, y_pred_binary)
auc_score = metrics.roc_auc_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("AUC Score:", auc_score)

Epoch 1/4
8028/8028 [==============================] - 15s 2ms/step - loss: 0.5910 - accuracy: 0.6633
Epoch 2/4
8028/8028 [==============================] - 13s 2ms/step - loss: 0.5734 - accuracy: 0.6845
Epoch 3/4
8028/8028 [==============================] - 12s 2ms/step - loss: 0.5725 - accuracy: 0.6856
Epoch 4/4
4014/4014 [==============================] - 3s 795us/step
Precision: 0.7798591023607353
Recall: 0.5191955272557851
F1 Score: 0.6233754125566392
AUC Score: 0.7308295540312849
